In [9]:
import datetime
import sys

print(f"UTC now= '{datetime.datetime.utcnow().isoformat().split('.')[0]}'")
print(f'Virtualenv used: {sys.executable}')

import importlib


import numpy as np
import pandas as pd
import ast
import pickle
import math

from pathlib import Path

from sklearn.cluster import KMeans

#for text pre-processing
import re, string

from sklearn.decomposition import TruncatedSVD
# bag of words
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.tree import DecisionTreeClassifier
#for model-building
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix,roc_curve,auc
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import tree

from sentence_transformers import SentenceTransformer
sentence_bert_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

import spacy
importlib.reload(spacy)
spacy_nlp_model = spacy.load('en_core_web_lg')

from scipy.optimize import minimize_scalar

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Initialize the lemmatizer
wl = WordNetLemmatizer()

from nltk.corpus import wordnet, stopwords

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

# for LIME import necessary packages
from lime.lime_text import LimeTextExplainer
from sklearn.pipeline import make_pipeline
from random import randint

from typing import List, Optional




UTC now= '2022-05-14T14:23:16'
Virtualenv used: /usr/bin/python3


[nltk_data] Downloading package punkt to
[nltk_data]     /home/mmozolewski/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mmozolewski/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/mmozolewski/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/mmozolewski/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/mmozolewski/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [8]:
def _save_pickle(df, outfile, results_folder):
  if results_folder:
    results_folder_local = f"{str(Path('~').expanduser().resolve())}/{results_folder}"
    # results_folder_local = results_folder
  else:
    results_folder_local = f"{str(Path('~').expanduser().resolve())}/data"
  Path(results_folder_local).mkdir(parents=True, exist_ok=True)
  fname = f"{results_folder_local}/{outfile}.pickle"
  print(fname)
  with open(fname, 'wb') as f:
    pickle.dump(df, f)


def get_LSA(df:pd.DataFrame, column_to_omit:str, variance:float):

  def get_LSA_variance(how_many_dimension):
    how_many_dimension = math.floor(how_many_dimension)
    svd = TruncatedSVD(how_many_dimension)
    normalizer = Normalizer(copy=False)
    lsa = make_pipeline(svd, normalizer)
    lsa.fit_transform(_df)
    return svd.explained_variance_ratio_.sum()*100

  def get_LSA_delta(how_many_dimension):
    explained_variance = get_LSA_variance(how_many_dimension)
    delta = round(math.fabs(explained_variance - variance), 0)
    # print(f"how_many_dimension= {how_many_dimension}, explained_variance= {explained_variance}, delta= {delta}")
    return delta

  def optimize_how_many_dimension(bounds):
    res = minimize_scalar(get_LSA_delta, bounds=bounds, method='bounded')
    # print(res.x)
    how_many_dimension = math.floor(res.x)
    if get_LSA_variance(how_many_dimension) < variance:
      how_many_dimension += 1
    return (how_many_dimension-1, get_LSA_variance(how_many_dimension))

  _df = df.loc[:, (df.columns != column_to_omit) & (df.columns != 'cluster_kmeans')]
  variables_max_count = len(_df.columns) - 1
  how_many_dimension, explained_variance = optimize_how_many_dimension(bounds=(1, variables_max_count/4))
  if explained_variance < variance:
    print(f"variance is to low: {explained_variance}")
    how_many_dimension, explained_variance = optimize_how_many_dimension(bounds=(how_many_dimension+1, variables_max_count/2))
  if explained_variance < variance:
    print(f"variance is to low: {explained_variance}")
    how_many_dimension, explained_variance = optimize_how_many_dimension(bounds=(how_many_dimension+1, variables_max_count))

  while get_LSA_variance(how_many_dimension) < variance:
    how_many_dimension += 1

  svd = TruncatedSVD(how_many_dimension)
  normalizer = Normalizer(copy=False)
  lsa = make_pipeline(svd, normalizer)
  _df_lsa = pd.DataFrame(lsa.fit_transform(_df))
  print(f"explained variance perc = {svd.explained_variance_ratio_.sum()*100}%")

  if 'cluster_kmeans' in df.columns:
    _df_lsa = pd.concat([df[['article_which_cities', 'cluster_kmeans']], _df_lsa], axis=1, join="inner")
  else:
    _df_lsa = pd.concat([df[['article_which_cities']], _df_lsa], axis=1, join="inner")
  # df_edges_lsa['cluster_kmeans'] = -1
  return _df_lsa

# get_LSA(df_edges_one_hot, 'article_which_cities', 95.0)


def k_means(df, column_to_ommit, how_many_clusters):
  kmeans = KMeans(n_clusters=how_many_clusters, random_state=2022).fit(df.loc[:, (df.columns != column_to_ommit) & (df.columns != 'cluster_kmeans')])
  # kmeans.labels_
  # print(pd.DataFrame(kmeans.cluster_centers_).reset_index())
  # print(pd.DataFrame(kmeans.cluster_centers_).reset_index()['index'])
  df['cluster_kmeans'] = kmeans.labels_
  # df_only_one_hots = df.loc[:, (df.columns != 'article_which_cities') & (df.columns != 'cluster_kmeans')]
  # df[['article_which_cities', 'cluster_kmeans']]
  return df


def get_TSNE_and_PCA_embeddings(df_edges, column_to_ommit):
  df_edges_independent_vars = df_edges.loc[:, (df_edges.columns != column_to_ommit) & (df_edges.columns != 'cluster_kmeans')]

  df_tsne = pd.DataFrame(TSNE(n_components=2).fit_transform(df_edges_independent_vars))
  df_tsne['cluster'] = df_edges['cluster_kmeans']
  df_tsne.columns = ['x1', 'x2', 'cluster']
  df_tsne['article_which_cities'] = df_edges['article_which_cities']

  df_pca = pd.DataFrame(PCA(n_components=2).fit_transform(df_edges_independent_vars))
  df_pca['cluster'] = df_edges['cluster_kmeans']
  df_pca.columns = ['x1', 'x2', 'cluster']
  df_pca['article_which_cities'] = df_edges['article_which_cities']

  scaler = MinMaxScaler()
  df_tsne[['x1', 'x2']] = scaler.fit_transform(df_tsne[['x1', 'x2']])
  df_pca[['x1', 'x2']] = scaler.fit_transform(df_pca[['x1', 'x2']])

  return df_tsne, df_pca


def wrap_by_word(s, n):
  '''returns a string where \\n is inserted between every n words'''
  if isinstance(s, str):
    a = s.split()
    ret = ''
    for i in range(0, len(a), n):
      ret += ' '.join(a[i:i+n]) + '\n'

    return ret
  return ""

# x = wrap_by_word('There is a dog and fox fighting in the park and there is an apple falling down.', 4)
# print(x)
def get_article_info(df_with_article_which_cities_column, column):
  df_nodes_cp = df_nodes.copy()
  df_nodes_cp['article_which_cities'] = df_nodes_cp['ID']#.apply(lambda x: f"{x:02d}")
  df_merged = df_with_article_which_cities_column.merge(df_nodes_cp, on='article_which_cities', how='left')

  return df_merged['article_which_cities'] + " (" + df_merged['Year'].apply(lambda x: str(int(x)) if not np.isnan(x) else "") + ") " + ": \n" + df_merged[column].apply(lambda x: wrap_by_word(x, 3))


def label_point(x, y, val, ax, factor):
  a = pd.concat({'x': x, 'y': y, 'val': val}, axis=1)
  for i, point in a.iterrows():
    ax.text(point['x'] + 1*factor, point['y'] - 10*factor, str(point['val']))


def visualise_clusters(df_2d_embedding, factor=1/200, base_size=6):
  _fig, _ax = plt.subplots(1, 1, figsize=(base_size*4,base_size*2))
  sns.scatterplot(data=df_2d_embedding, x='x1', y='x2', hue='cluster', style='cluster', legend="full", alpha=0.9, s=120, palette="deep", ax=_ax)
  label_point(df_2d_embedding['x1'], df_2d_embedding['x2'], get_article_info(df_2d_embedding, column='Title'), _ax, factor)
  # display(_fig)

# get_article_info(df_edges_lsa_tsne, column='Title')

In [13]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics

from sklearn.cluster import KMeans, MiniBatchKMeans

import logging
from optparse import OptionParser
import sys
from time import time


# Display progress logs on stdout
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")

# parse commandline arguments
op = OptionParser()
op.add_option(
  "--lsa",
  dest="n_components",
  type="int",
  help="Preprocess documents with latent semantic analysis.",
)
op.add_option(
  "--no-minibatch",
  action="store_false",
  dest="minibatch",
  default=True,
  help="Use ordinary k-means algorithm (in batch mode).",
)
op.add_option(
  "--no-idf",
  action="store_false",
  dest="use_idf",
  default=True,
  help="Disable Inverse Document Frequency feature weighting.",
)
op.add_option(
  "--use-hashing",
  action="store_true",
  default=False,
  help="Use a hashing feature vectorizer",
)
op.add_option(
  "--n-features",
  type=int,
  default=10000,
  help="Maximum number of features (dimensions) to extract from text.",
)
op.add_option(
  "--verbose",
  action="store_true",
  dest="verbose",
  default=False,
  help="Print progress reports inside k-means algorithm.",
)

print(__doc__)


def is_interactive():
  return not hasattr(sys.modules["__main__"], "__file__")


if not is_interactive():
  op.print_help()
  print()

# work-around for Jupyter notebook and IPython console
argv = [] if is_interactive() else sys.argv[1:]
(opts, args) = op.parse_args(argv)
if len(args) > 0:
  op.error("this script takes no arguments.")
  sys.exit(1)


categories = [
  "alt.atheism",
  "talk.religion.misc",
  "comp.graphics",
  "sci.space",
]

dataset = fetch_20newsgroups(
  subset="all", categories=categories, shuffle=True, random_state=42
)

print("%d documents" % len(dataset.data))
print("%d categories" % len(dataset.target_names))


Automatically created module for IPython interactive environment
3387 documents
4 categories


In [14]:
labels = dataset.target
true_k = np.unique(labels).shape[0]

print("Extracting features from the training dataset using a sparse vectorizer")
t0 = time()
if opts.use_hashing:
  if opts.use_idf:
    # Perform an IDF normalization on the output of HashingVectorizer
    hasher = HashingVectorizer(
      n_features=opts.n_features,
      stop_words="english",
      alternate_sign=False,
      norm=None,
    )
    vectorizer = make_pipeline(hasher, TfidfTransformer())
  else:
    vectorizer = HashingVectorizer(
      n_features=opts.n_features,
      stop_words="english",
      alternate_sign=False,
      norm="l2",
    )
else:
  vectorizer = TfidfVectorizer(
    max_df=0.5,
    max_features=opts.n_features,
    min_df=2,
    stop_words="english",
    use_idf=opts.use_idf,
  )
X = vectorizer.fit_transform(dataset.data)

print("done in %fs" % (time() - t0))
print("n_samples: %d, n_features: %d" % X.shape)
print()

if opts.n_components:
  print("Performing dimensionality reduction using LSA")
  t0 = time()
  # Vectorizer results are normalized, which makes KMeans behave as
  # spherical k-means for better results. Since LSA/SVD results are
  # not normalized, we have to redo the normalization.
  svd = TruncatedSVD(opts.n_components)
  normalizer = Normalizer(copy=False)
  lsa = make_pipeline(svd, normalizer)

  X = lsa.fit_transform(X)

  print("done in %fs" % (time() - t0))

  explained_variance = svd.explained_variance_ratio_.sum()
  print(
    "Explained variance of the SVD step: {}%".format(int(explained_variance * 100))
  )

Extracting features from the training dataset using a sparse vectorizer
done in 0.437999s
n_samples: 3387, n_features: 10000



In [15]:
if opts.minibatch:
  km = MiniBatchKMeans(
    n_clusters=true_k,
    init="k-means++",
    n_init=1,
    init_size=1000,
    batch_size=1000,
    verbose=opts.verbose,
  )
else:
  km = KMeans(
    n_clusters=true_k,
    init="k-means++",
    max_iter=100,
    n_init=1,
    verbose=opts.verbose,
  )

print("Clustering sparse data with %s" % km)
t0 = time()
km.fit(X)
print("done in %0.3fs" % (time() - t0))

Clustering sparse data with MiniBatchKMeans(batch_size=1000, init_size=1000, n_clusters=4, n_init=1,
                verbose=False)
done in 0.042s


In [22]:
print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels, km.labels_))
print("Completeness: %0.3f" % metrics.completeness_score(labels, km.labels_))
print("V-measure: %0.3f" % metrics.v_measure_score(labels, km.labels_))
print("Adjusted Rand-Index: %.3f" % metrics.adjusted_rand_score(labels, km.labels_))
print(
  "Silhouette Coefficient: %0.3f"
  % metrics.silhouette_score(X, km.labels_, sample_size=1000)
)

if not opts.use_hashing:
  print("Top terms per cluster:")

  if opts.n_components:
    original_space_centroids = svd.inverse_transform(km.cluster_centers_)
    order_centroids = original_space_centroids.argsort()[:, ::-1]
  else:
    order_centroids = km.cluster_centers_.argsort()[:, ::-1]

  # terms = vectorizer.get_feature_names_out()
  terms = list(vectorizer.vocabulary_.keys())
  # print(terms)
  for i in range(true_k):
    print("Cluster %d:" % i, end="")
    for ind in order_centroids[i, :10]:
      print(" %s" % terms[ind], end="")
    print()

Homogeneity: 0.501
Completeness: 0.565
V-measure: 0.531
Adjusted Rand-Index: 0.464
Silhouette Coefficient: 0.008
Top terms per cluster:
Cluster 0: funded compact permission prototype forbid fastest format silicon parents arrays
Cluster 1: neural unbelievers photoshop innocence spacelab current blackbody fastest brings ics
Cluster 2: cd implementation lcd sigh ch981 evening imagination fewer joint unbelievers
Cluster 3: wang unbelievers rewrite altered askew foo damage current galilean emory
